In [1]:
import numpy as np
import cmath, pygame, sys

pygame 2.0.0 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Inspired by FakeNameSe's __[repo](https://github.com/FakeNameSE/Boids-with-obstacles-and-goals/blob/master/experiments/boids-with-predators.py)__

In [14]:
class Animal(pygame.sprite.DirtySprite):
    
    def __init__(self,x,y,FOV,pred,side_length):
        
        pygame.sprite.DirtySprite.__init__(self)
        
        self.image = pygame.Surface((side_length,side_length))
        if pred:
            self.image.fill((255,0,0))
        else:
            self.image.fill((0,255,0))
        self.rect = self.image.get_rect()
        
        self.rect.x = x
        self.rect.y = y
        self.FOVangle = FOV
        self.velocityX = np.random.uniform(-maxvel/(2*np.sqrt(2)),maxvel/(2*np.sqrt(2)))
        self.velocityY = np.random.uniform(-maxvel/(2*np.sqrt(2)),maxvel/(2*np.sqrt(2)))
        
        self.pred = pred
        self.alive = True
        
        if self.velocityX == 0 and self.velocityY > 0:
            self.orient = np.pi/2
        elif self.velocityX == 0 and self.velocityY < 0:
            self.orient = -np.pi/2
        else:
            self.orient = cmath.polar(complex(self.velocityX,self.velocityY))[1]
        
        self.dirty = 1
        
    def distance(self,animal):
        
        return np.sqrt((self.rect.x-animal.rect.x)**2+(self.rect.y-animal.rect.y)**2) #Euclidean distance
    
    def FOV(self):
        
        return (self.orient-self.FOVangle/2,self.orient+self.FOVangle/2)
    
    def angle(self, animal):
        if self.rect.x == animal.rect.x and animal.rect.y>self.rect.y:
            return np.pi/2
        elif self.rect.x == animal.rect.x and animal.rect.y<self.rect.y:
            return -np.pi/2
        else:
            return cmath.polar(complex(animal.rect.x-self.rect.x,animal.rect.y-self.rect.y))[1]
    
    #Find out which animals you can see (for interspecific)
    def see(self,animal_list):
        
        visible_animals = []
        
        for animal in animal_list:
            angle = self.angle(animal)
            FOV = self.FOV()
            
            if angle > FOV[0] and angle < FOV[1]:
                visible_animals.append(animal)
                
        return visible_animals
    
    def sense(self,animal_list):
        sensed_animals = []
        
        for animal in animal_list:
            dist = self.distance(animal)
            if dist < max_dist:
                sensed_animals.append(animal)
        
        return sensed_animals
    
    #Find out which animals are same as you
    def classify(self,animal_list):
        
        conspecifics = []
        heterospecifics = []
        
        for animal in animal_list:
            if self.pred == animal.pred:
                conspecifics.append(animal)
            else:
                heterospecifics.append(animal)
        
        return conspecifics, heterospecifics
    
    #following three functions are standard boid-like flocking algorithms 
    def move_closer(self,cons):
        
        x_avg = 0
        y_avg = 0
        
        for con in cons:
            if self.rect.x == con.rect.x and self.rect.y == con.rect.y:
                continue
            
            x_avg += (self.rect.x-con.rect.x)
            y_avg += (self.rect.y-con.rect.y)
        
        x_avg /= len(cons)
        y_avg /= len(cons)
        
        self.velocityX -= x_avg/cohesion_weight
        self.velocityY -= y_avg/cohesion_weight
        
    def move_with(self,cons):
        
        velx_avg = 0
        vely_avg = 0
        
        for con in cons:
            if self.rect.x == con.rect.x and self.rect.y == con.rect.y:
                continue
            
            velx_avg += con.velocityX
            vely_avg += con.velocityY
        
        velx_avg /= len(cons)
        vely_avg /= len(cons)
        
        self.velocityX += velx_avg/adhesion_weight
        self.velocityY += vely_avg/adhesion_weight
        
    def dont_crowd(self,cons):
        
        x_dist = 0
        y_dist = 0
        
        num_close = 0
        
        for con in cons:
            
            dist = self.distance(con)
            
            if dist < min_dist:
                num_close += 1
                
                xdiff = self.rect.x - con.rect.x
                ydiff = self.rect.y - con.rect.y
                
                if xdiff >= 0:
                    xdiff = np.sqrt(min_dist) - xdiff
                elif xdiff < 0:
                    xdiff = -np.sqrt(min_dist) - xdiff

                if ydiff >= 0:
                    ydiff = np.sqrt(min_dist) - ydiff
                elif ydiff < 0:
                    ydiff = -np.sqrt(min_dist) - ydiff

                x_dist += xdiff
                y_dist += ydiff

        if num_close == 0:
            return
        
        self.velocityX -= x_dist/seperation_weight
        self.velocityY -= y_dist/seperation_weight
    
    def flock(self,cons):
        self.move_closer(cons)
        self.move_with(cons)
        self.dont_crowd(cons)
    
    #Prey should try to run away from threats
    
    def scared(self,pred_list):
        
        scared = False
        for pred in pred_list:
            if self.distance(pred) < scare_distance:
                scared = True
        
        return scared
    
    def run_away(self,predator):
        
        self.velocityX += -(((predator.rect.x + (2 * predator.velocityX)) - self.rect.x) /pred_run_weight)
        self.velocityY += -(((predator.rect.y + (2 * predator.velocityY)) - self.rect.y) /pred_run_weight)
    
    #Predators should try to hunt down prey by predicting where prey will go
    def chase(self,prey_list):
        
        dists = []
        prey_COMx = 0
        prey_COMy = 0
        
        #Calculate COM of prey
        for prey in prey_list:
            prey_COMx += prey.rect.x
            prey_COMy += prey.rect.y
        
        prey_COMx /= len(prey_list)
        prey_COMy /= len(prey_list)
        
        #Find out how far each individual is from the COM
        for prey in prey_list:
            x_dist = prey_COMx - prey.rect.x
            y_dist = prey_COMy - prey.rect.y
            
            dist = np.sqrt((x_dist)**2 + (y_dist)**2)
            dists.append(dist)
        
        #Look for the individual that's on the fringes of the flock
        focal_prey = prey_list[np.where(dists == max(dists))[0][0]]
        
        
        del dists #To save memory
        
        self.velocityX += (focal_prey.rect.x - self.rect.x + 2*(focal_prey.velocityX))/chase_weight
        self.velocityY += (focal_prey.rect.y - self.rect.y + 2*(focal_prey.velocityY))/chase_weight
        
        del focal_prey #To save memory
     
    
    #Actual movement based on the velocities
    def move(self,wrap):
        if abs(self.velocityX) > maxvel or abs(self.velocityY) > maxvel:
            scaleFactor = maxvel / max(abs(self.velocityX), abs(self.velocityY))
            self.velocityX *= scaleFactor
            self.velocityY *= scaleFactor
        
        self.rect.x += self.velocityX + np.random.uniform(0,1)
        self.rect.y += self.velocityY + np.random.uniform(0,1)
        if wrap:
            self.rect.x = self.rect.x%(width-25)
            self.rect.y = self.rect.y%(height-25)
        else:
            if self.rect.x > width or self.rect.x < 10:
                self.velocityX *= -1
            if self.rect.y > height or self.rect.y < 10:
                self.velocityY *= -1
        
        
        #Update your orientation
        if self.velocityX == 0 and self.velocityY > 0:
            self.orient = np.pi/2
        elif self.velocityX == 0 and self.velocityY < 0:
            self.orient = -np.pi/2
        else:
            self.orient = cmath.polar(complex(self.velocityX,self.velocityY))[1]
        
        self.dirty = 1

In [34]:
#Constants

cohesion_weight = 11
adhesion_weight = 11
seperation_weight = 10
maxvel = 7
chase_weight = 9
pred_run_weight=0.2

#max_dist is the maximum distance upto which animals can be sensed. min_dist is the minimum distance b/w conspecifics
#scare_dist is the distance at which the prey starts trying to evade the predators
scare_distance = 100
max_dist = 600
min_dist = 30

#The FOV is the maximum possible angle that can be subtended in the eye of the animal
pred_FOV = 2*np.pi/3
prey_FOV = 4*np.pi/3

num_prey = 170
num_predators = 10
pred_size = 30
prey_size = 10

bg_color = (0,0,0)


In [36]:
pygame.init()
width, height = pygame.display.Info().current_w, pygame.display.Info().current_h - 40
screen = pygame.display.set_mode((width,height))

def eaten(sprite1, sprite2):
    collided = pygame.sprite.collide_mask(sprite1, sprite2)
    return collided

pygame.display.set_caption("Flocking with predators")

background = pygame.Surface(screen.get_size())
background = background.convert()
background.fill(bg_color)

all_animals = pygame.sprite.LayeredDirty()

for i in range(0,num_prey):
    prey = Animal(np.random.randint(0,width),np.random.randint(0,height),prey_FOV,False,prey_size)
    all_animals.add(prey)
    
for i in range(0,num_predators):
    pred = Animal(np.random.randint(0,width),np.random.randint(0,height),pred_FOV,True,pred_size)
    all_animals.add(pred)
    
clock = pygame.time.Clock()
running = True

all_animals.clear(screen,background)


#Main loop for running the simulation
running = True
while running:  
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
    
    fps = "FPS: {0:.2f}".format(clock.get_fps())
    pygame.display.set_caption(fps)
    
    for animal in all_animals:
 
        sensed_animals = animal.sense(all_animals)
        conspecifics, heterospecifics = animal.classify(sensed_animals)
        hunted = pygame.sprite.Group()
        
        if len(conspecifics) > 1:
            animal.flock(conspecifics)
        
        heterospecifics = animal.see(heterospecifics)
        
        if len(heterospecifics) > 0:
            if animal.pred:
                for prey in heterospecifics:
                    hunted.add(prey)
                animal.chase(heterospecifics)
            else:
                if animal.scared(heterospecifics):
                    for predator in heterospecifics:
                        animal.run_away(predator)
        
        animal.move(True)
        if len(hunted) > 0:
            for prey in hunted:
                if eaten(animal,prey):
                        all_animals.remove(prey)
    
    
    rects = all_animals.draw(screen)
    pygame.display.update(rects)
    
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 